In [ ]:
import math
import highspy

In [ ]:
import marimo as mo
import nbformat
import util

# MILP 定式化 (HiGHS)

数理最適化モデルを用いて SCSP を解く.
定式化については SCIP 版を参照.

In [ ]:
class Model:
    def __init__(self, instance: list[str]):
        max_len = sum(len(s) for s in instance)

        highs = highspy.Highs()

        seqs = [
            [
                highs.addIntegral(lb=0, ub=max_len - 1)
                for idx, _ in enumerate(s)
            ]
            for s in instance
        ]
        for seq in seqs:
            for idx, _ in enumerate(seq):
                if idx == 0:
                    continue
                highs.addConstr(seq[idx - 1] + 1 <= seq[idx])

        for idx1, (s1, seq1) in enumerate(zip(instance, seqs)):
            for idx2, (s2, seq2) in enumerate(zip(instance, seqs)):
                if idx1 >= idx2:
                    continue

                for cidx1, (c1, cvar1) in enumerate(zip(s1, seq1)):
                    for cidx2, (c2, cvar2) in enumerate(zip(s2, seq2)):
                        if c1 != c2:
                            lt = highs.addBinary()
                            gt = highs.addBinary()
                            highs.addConstr(lt + gt == 1)
                            big_m = max_len
                            highs.addConstr(cvar1 + 1 <= cvar2 + big_m * (1 - lt))
                            highs.addConstr(cvar1 + big_m * (1 - gt) >= cvar2 + 1)

        obj = highs.addVariable(lb=0, ub=max_len)
        for seq in seqs:
            highs.addConstr(obj >= seq[-1])
        highs.setObjective(obj=obj, sense=highspy.ObjSense.kMinimize)

        self.instance = instance
        self.highs = highs
        self.seqs = seqs

    def solve(self, time_limit: int | None = 60) -> "Model":
        if time_limit is not None:
            self.highs.setOptionValue("time_limit", time_limit)
        self.highs.solve()
        return self

    def to_solution(self) -> str | None:
        info = self.highs.getInfo()
        primal_status = info.primal_solution_status
        if primal_status != highspy.SolutionStatus.kSolutionStatusFeasible:
            return None

        highssolution = self.highs.getSolution()
        objval = int(self.highs.getObjectiveValue())
        sol_char_idx = 0
        solution = ""
        while sol_char_idx <= objval:
            found = False
            for idx, (s, seq) in enumerate(zip(self.instance, self.seqs)):
                for c_idx, cvar in enumerate(seq):
                    if math.isclose(
                        highssolution.col_value[cvar.index],
                        sol_char_idx
                    ):
                        solution += s[c_idx]
                        found = True
                        sol_char_idx += 1
                if found:
                    break
            if not found:
                sol_char_idx += 1

        return solution

In [ ]:
def solve(instance: list[str], time_limit: int | None = 60) -> str | None:
    return Model(instance).solve(time_limit).to_solution()

In [ ]:
instance_01 = util.parse("uniform_q26n004k015-025.txt")
solution_01 = solve(instance_01)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 7665 rows; 5139 cols; 20384 nonzeros; 5138 integer variables (5054 binary)
Coefficient ranges:
  Matrix [1e+00, 8e+01]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 8e+01]
  RHS    [1e+00, 8e+01]


Presolving model
5138 rows, 2612 cols, 15330 nonzeros  0s


4987 rows, 2612 cols, 14877 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   4987 rows
   2612 cols (2527 binary, 84 integer, 1 implied int., 0 continuous, 0 domain fixed)
   14877 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   24              inf                  inf        0      0      0         0     0.1s


...

      6885       0         0   0.00%   25              74                66.22%       76      0      0    281612    43.1s


      6885       0         0   0.00%   25              74                66.22%       76     50     23    281728    43.1s


      7826      94       301   2.78%   25              74                66.22%    33031    983   2279    330670    48.4s


      8051     125       379   2.78%   25              74                66.22%     1109    918   2786    347501    53.5s


      8447     158       511   2.78%   25              74                66.22%     1003    899   3649    380350    60.0s
      8447     158       511   2.78%   25              74                66.22%     1003    899   3649    380350    60.0s

Solving report
  Status            Time limit reached
  Primal bound      74
  Dual bound        25
  Gap               66.22% (tolerance: 0.01%)
  P-D integral      39.8981656145
  Solution status   feasible
                    74 (objective)
                    0 (bound viol.)
                    7.1054273576e-15 (int. viol.)
                    0 (row viol.)
  Timing            60.03 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 5
  Nodes             8447
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     380350 (total)
                    1365 (strong br.)
                    38340 (separation)
                    66052 (heuristics)


In [ ]:
_instance = instance_01
_solution = solution_01

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 75) ---
 Sol: tkgnkuhmpxnhtqgxzvxulcinycosovozppljigqfolnbxxecvsuaqpvzissgbxrddbcsvrvnngf
str1: tkgnkuhmpxnhtqgxzvx---i----s-----------------------------------------------
str2: ----------------------i---o--------ji-qfolnbxx-cvsu-qpv-iss-bx------------f
str3: -----u--------------lcinycosovozppl------------------p---------------------
str4: ----------------------i--------------g--------e-v--a---z---gb-rddbcsvrvnngf

solution is feasible: True


In [ ]:
instance_02 = util.parse("uniform_q26n008k015-025.txt")
solution_02 = solve(instance_02)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 38584 rows; 25782 cols; 102774 nonzeros; 25781 integer variables (25606 binary)
Coefficient ranges:
  Matrix [1e+00, 2e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+02]
  RHS    [1e+00, 2e+02]


Presolving model


25781 rows, 12979 cols, 77168 nonzeros  0s


25096 rows, 12979 cols, 75113 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   25096 rows
   12979 cols (12803 binary, 175 integer, 1 implied int., 0 continuous, 0 domain fixed)
   75113 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   24              inf                  inf        0      0      0         0     0.6s


 R       0       0         0   0.00%   24              147               83.67%        0      0      0      6906     2.5s


         0       0         0   0.00%   25              147               82.99%    22513   4366    185     34501    25.7s


       101      71        15   0.00%   25              147               82.99%    24390   4392    234     62374    60.1s
       101      71        15   0.00%   25              147               82.99%    24390   4392    234     62374    60.1s

Solving report
  Status            Time limit reached
  Primal bound      147
  Dual bound        25
  Gap               82.99% (tolerance: 0.01%)
  P-D integral      47.807171215
  Solution status   feasible
                    147 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            60.06 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 2
  Nodes             101
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     62374 (total)
                    0 (strong br.)
                    6669 (separation)
                    39468 (heuristics)


In [ ]:
_instance = instance_02
_solution = solution_02

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 148) ---
 Sol: ulcinygevacosovozpplrxwtknbczfjxqkrdrlctodtmprpxwgnkuhmpxnhvxerzbrvigpledcvdpfzsmsbroqvbbhtcsvrvnngyfqgxzplrzxucpmqvgtdfuxisolnbxxcvsuqpvissbxfcdsbo
str1: -----------------------tk------------------------gnkuhmpxnh-------------------------------t----------qgxz----------v-----xis------------------------
str2: ---i-------o------------------j------------------------------------i-----------------q--------------f-----------------------olnbxxcvsuqpvissbxf-----
str3: ulciny----cosovozppl------------------------p-------------------------------------------------------------------------------------------------------
str4: ---i--geva------z--------------------------------g--------------br------d--

In [ ]:
instance_03 = util.parse("uniform_q26n016k015-025.txt")
solution_03 = solve(instance_03)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 140897 rows; 94040 cols; 375510 nonzeros; 94039 integer variables (93716 binary)
Coefficient ranges:
  Matrix [1e+00, 3e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 3e+02]
  RHS    [1e+00, 3e+02]


Presolving model


94039 rows, 47182 cols, 281794 nonzeros  0s


91182 rows, 47182 cols, 273223 nonzeros  1s


Objective function is integral with scale 1

Solving MIP model with:
   91182 rows
   47182 cols (46858 binary, 323 integer, 1 implied int., 0 continuous, 0 domain fixed)
   273223 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   25              inf                  inf        0      0      0         0     3.9s


         0       0         0   0.00%   25              inf                  inf        0      0      6     24836    27.1s


         0       0         0   0.00%   25              inf                  inf   288925  11617      7     44359    56.9s


         0       0         0   0.00%   25              inf                  inf    95173  14460      8     46635    60.1s

Solving report
  Status            Time limit reached
  Primal bound      inf
  Dual bound        25
  Gap               inf
  P-D integral      0
  Solution status   -
  Timing            60.07 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 0
  Nodes             0
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     46635 (total)
                    0 (strong br.)
                    21799 (separation)
                    0 (heuristics)


In [ ]:
_instance = instance_03
_solution = solution_03

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution not found ---


In [ ]:
instance_04 = util.parse("uniform_q05n010k010-010.txt")
solution_04 = solve(instance_04)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 10945 rows; 7331 cols; 29120 nonzeros; 7330 integer variables (7230 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+02]
  RHS    [1e+00, 1e+02]
Presolving model


7330 rows, 3716 cols, 21890 nonzeros  0s


6964 rows, 3716 cols, 20792 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   6964 rows
   3716 cols (3615 binary, 100 integer, 1 implied int., 0 continuous, 0 domain fixed)
   20792 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   10              inf                  inf        0      0      0         0     0.3s


...

       909     750        64   0.17%   10.00010305     55                81.82%     8214   1357    783    132237    32.9s


 L    1007     783        85   0.20%   10.00010305     51                80.39%    12461   1477    883    137755    37.8s


      1270     864       129   0.20%   10.00010305     51                80.39%    13087   1541   1064    182778    45.1s


      1396     953       146   0.20%   10.00010305     51                80.39%     1832   1647   1261    208471    52.2s


      1406     953       150   0.20%   10.00010305     51                80.39%     1890   1677   1310    228086    60.0s
      1406     953       150   0.20%   10.00010305     51                80.39%     1890   1677   1310    228086    60.0s

Solving report
  Status            Time limit reached
  Primal bound      51
  Dual bound        11
  Gap               78.43% (tolerance: 0.01%)
  P-D integral      42.6274600134
  Solution status   feasible
                    51 (objective)
                    0 (bound viol.)
                    2.79776202206e-14 (int. viol.)
                    0 (row viol.)
  Timing            60.03 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 6
  Nodes             1406
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     228086 (total)
                    0 (strong br.)
                    6671 (separation)
                    154884 (heuristics)


In [ ]:
_instance = instance_04
_solution = solution_04

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 52) ---
  Sol: dcbacdebadbaaeaebadacbeeadbccdeaeeadcbcdeedacbdbecde
str01: dcb-c---------------c----dbcc-e---------------------
str02: --b--d---db--e-e------ee--b--d----------------------
str03: -c-acde------e------c-e---b---e---------------------
str04: ---a--e--d--------d------d---de------b-d--d---------
str05: ---ac--b-----e-e----c---a-bc--e---------------------
str06: --b----ba-b--e--b-d-cb--a---------------------------
str07: --b----ba----eaebada--------------------------------
str08: ------e------e-e------e----c---------b-d-----b--e--e
str09: -c--cde------e----da-----d-c-d----------------------
str10: --b--d--a-b-------d--be-a------a---d----------------

solution is feasible: True


In [ ]:
instance_05 = util.parse("uniform_q05n050k010-010.txt")
solution_05 = solve(instance_05)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 294593 rows; 196563 cols; 785248 nonzeros; 196562 integer variables (196062 binary)
Coefficient ranges:
  Matrix [1e+00, 5e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 5e+02]
  RHS    [1e+00, 5e+02]


Presolving model


196562 rows, 98532 cols, 589186 nonzeros  0s


186760 rows, 98532 cols, 559780 nonzeros  2s


Objective function is integral with scale 1

Solving MIP model with:
   186760 rows
   98532 cols (98031 binary, 500 integer, 1 implied int., 0 continuous, 0 domain fixed)
   559780 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   10              inf                  inf        0      0      0         0     4.3s


         0       0         0   0.00%   10              inf                  inf        0      0      6     22092    60.1s

Solving report
  Status            Time limit reached
  Primal bound      inf
  Dual bound        10
  Gap               inf
  P-D integral      0
  Solution status   -
  Timing            60.08 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 0
  Nodes             0
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     22092 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)


In [ ]:
_instance = instance_05
_solution = solution_05

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution not found ---


In [ ]:
instance_06 = util.parse("nucleotide_n010k010.txt")
solution_06 = solve(instance_06)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 10117 rows; 6779 cols; 26912 nonzeros; 6778 integer variables (6678 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+02]
  RHS    [1e+00, 1e+02]
Presolving model


6778 rows, 3440 cols, 20234 nonzeros  0s


6450 rows, 3440 cols, 19250 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   6450 rows
   3440 cols (3339 binary, 100 integer, 1 implied int., 0 continuous, 0 domain fixed)
   19250 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   10              inf                  inf        0      0      0         0     0.2s


...

      1945      80       119   0.02%   11              31                64.52%     6486    205    694    190357    36.0s


      2159     111       174   0.02%   11              31                64.52%     2104   1452   1027    224678    42.8s


      2252     122       196   0.02%   11              31                64.52%     2342   1482   1189    257887    51.5s


      2468     143       260   0.02%   11              31                64.52%     5188    299   1639    286110    57.7s


      2561     154       293   0.02%   11              31                64.52%    18745   1465   1829    298407    60.0s

Solving report
  Status            Time limit reached
  Primal bound      31
  Dual bound        11
  Gap               64.52% (tolerance: 0.01%)
  P-D integral      38.7559191488
  Solution status   feasible
                    31 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            60.01 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 5
  Nodes             2561
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     298407 (total)
                    0 (strong br.)
                    17149 (separation)
                    127627 (heuristics)


In [ ]:
_instance = instance_06
_solution = solution_06

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 32) ---
  Sol: ATGGGATACGCTTCCCACGGAACTTGCTGTAA
str01: ATGGGATACG----------------------
str02: AT---A--C-CTTCCC----------------
str03: --------C-------ACG-AA-TTG----A-
str04: -T---A-A--------A---A--T--CTGT--
str05: A-GG--TA--------AC--AA--------A-
str06: -T----T-C-CT----A-GG---T------A-
str07: -T----T--G-T----A-G-A--T--CT----
str08: -TGGGA-A-G-TTC------------------
str09: -T----T-C-C-----AC--AACT--------
str10: -T------C--T----A---AAC--G----AA

solution is feasible: True


In [ ]:
instance_07 = util.parse("nucleotide_n050k050.txt")
solution_07 = solve(instance_07)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 6853270 rows; 4569681 cols; 18273720 nonzeros; 4569680 integer variables (4567180 binary)


Coefficient ranges:
  Matrix [1e+00, 2e+03]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+03]
  RHS    [1e+00, 2e+03]


Presolving model


4569680 rows, 2286091 cols, 13706540 nonzeros  12s


4524372 rows, 2286091 cols, 13570616 nonzeros  61s


Presolve: Time limit reached

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0    62.4s

Solving report
  Status            Time limit reached
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  P-D integral      0
  Solution status   -
  Timing            62.44 (total)
                    0.00 (presolve)
    

In [ ]:
_instance = instance_07
_solution = solution_07

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

In [ ]:
instance_08 = util.parse("protein_n010k010.txt")
solution_08 = solve(instance_08)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 12853 rows; 8603 cols; 34208 nonzeros; 8602 integer variables (8502 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+02]
  RHS    [1e+00, 1e+02]
Presolving model


8602 rows, 4352 cols, 25706 nonzeros  0s


8220 rows, 4352 cols, 24560 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   8220 rows
   4352 cols (4251 binary, 100 integer, 1 implied int., 0 continuous, 0 domain fixed)
   24560 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   10              inf                  inf        0      0      0         0     0.1s


...

       631     390        81   0.00%   11              65                83.08%    13250   1972    676    125328    34.9s


       721     432       102   0.78%   11              65                83.08%     9230   2004    839    153274    41.5s


       937     503       153   0.78%   11              65                83.08%    13946   1736   1144    182757    47.4s


      1011     562       172   0.78%   11              65                83.08%     2195   1796   1233    198540    55.3s


      1028     563       180   0.78%   11              65                83.08%     2319   1813   1279    206489    60.1s

Solving report
  Status            Time limit reached
  Primal bound      65
  Dual bound        11
  Gap               83.08% (tolerance: 0.01%)
  P-D integral      49.9379967533
  Solution status   feasible
                    65 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            60.06 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 6
  Nodes             1028
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     206489 (total)
                    0 (strong br.)
                    6622 (separation)
                    137572 (heuristics)


In [ ]:
_instance = instance_08
_solution = solution_08

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 66) ---
  Sol: MALSYCNFDAIQSSLNAIEEHVNELHDPKGTLSYQHFRKAGVTHLVPGFNEAFGGYQNSRRPYQNG
str01: MALSYC---------------------PKGT-----------------------------------
str02: M----------QSSLNAI---------P-------------V------------------------
str03: M--------------------------P---LSYQHFRK---------------------------
str04: M-----------------EEHVNELHD---------------------------------------
str05: M--S--NFDAI--------------------------R-A----L---------------------
str06: M------F-----------------------------R-----------N------QNSR----NG
str07: M------F-------------------------Y-----A---H-------AFGGY----------
str08: M--S------------------------K-------F-----T----------------RRPYQ--
str09: M--S---F-------------V-----------------AGVT--------A----Q---------
str10: M-----

In [ ]:
instance_09 = util.parse("protein_n050k050.txt")
solution_09 = solve(instance_09)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 8643727 rows; 5763319 cols; 23048272 nonzeros; 5763318 integer variables (5760818 binary)


Coefficient ranges:
  Matrix [1e+00, 2e+03]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+03]
  RHS    [1e+00, 2e+03]


Presolving model


5763318 rows, 2882910 cols, 17287454 nonzeros  14s


5708453 rows, 2882910 cols, 17122859 nonzeros  75s


Presolve: Time limit reached

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0    76.7s

Solving report
  Status            Time limit reached
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  P-D integral      0
  Solution status   -
  Timing            76.66 (total)
                    0.00 (presolve)
    

In [ ]:
_instance = instance_09
_solution = solution_09

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS